In [ ]:
import geopandas as gpd
import pandas as pd

# Load election data
data_dir = "../data/"
election_data = pd.read_csv(data_dir + "G20192-Columnes-ME.csv", sep=';')

# Load GeoJSON file
map_catalonia='Límits administratius municipals de Catalunya.geojson'
geo_data = gpd.read_file(data_dir + map_catalonia)

# Translate
election_data.rename(columns={'Nom Municipi': 'municipality'}, inplace=True)


In [ ]:
geo_data.rename(columns={'nom_muni': 'municipality'}, inplace=True)
geo_data_sorted=geo_data.sort_values(by='municipality')


In [ ]:
votes_by_municipality = election_data.iloc[:, 21:].groupby(by=election_data["municipality"]).sum()
votes_by_municipality.reset_index()
votes_by_municipality

In [ ]:
winner_by_municipality=votes_by_municipality.idxmax(axis=1)
winner_by_municipality.reset_index()
list_winners=list(winner_by_municipality)
votes_by_municipality['winner']=list_winners
votes_by_municipality
votes_by_municipality.sort_values(by='municipality')

In [ ]:
merged_data = geo_data_sorted.merge(votes_by_municipality, on='municipality')
merged_data

In [ ]:
import plotly.express as px

fig = px.choropleth_mapbox(
    merged_data,
    geojson=merged_data.geometry,
    locations=merged_data.index,
    color='winner',  # Change to the column you want to visualize
    hover_name='municipality',
    mapbox_style="carto-positron",
    center={"lat": 41.8781, "lon": 1.7834},  # Center of Catalonia
    zoom=7
)

fig.update_geos(fitbounds="locations", visible=False)

fig.show()